**스타트캠프 교육 자료 저작권 및 사용 제한 안내**
- `스타트캠프 교육 자료의 모든 권리는 저작권자에게 있습니다.`
- `이 자료는 동의없이 외부로 유출하거나 개인적인 용도로 영리적인 목적을 위해 사용할 수 없습니다.`

## GPT 4o-mini 버전 API 실습 [Tool 만들기]
### Tool 기능이란?
+ GPT 모델의 기본 언어 생성 기능 외에 특정 작업을 수행하기 위해 특별히 설계된 도구를 사용할 수 있도록 해주는 확장 기능. ex) 이미지 생성, 인터넷 검색, 파이썬 코드 실행 등
+ GPT 모델은 특정 작업에 대해 더 정확하고 유용한 결과를 제공

### 1. 실시간 기상 정보를 알려주는 chatbot 만들기
+ 날씨 호출 tool 만들기


In [1]:
# 필요 라이브러리 설치
# !pip install openai

#API-KEY 설정
OPENAI_API_KEY = "OPENAI_API_KEY"
GPT_MODEL = 'gpt-4o-mini'

In [2]:
import json
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

#GPT API호출 및 응답 가져오기
def chat_completion_request(messages, tools=None, tool_choice=None, model=GPT_MODEL):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice=tool_choice,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e


In [ ]:
#Tool 선언
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",                #함수 이름
            "description": "현재 날씨 정보를 가져옵니다", #함수 설명
            "parameters": {                               #함수 파라미터 지정
                "type": "object",
                "properties": {
                    "location": {                         #파라미터 명
                        "type": "string",                 #파라미터 타입
                        "description": "도시 또는 지역을 입력하세요, 예: 서울, 충청도", #파라미터 설명 및 예시
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "사용할 온도 단위를 입력하세요. 사용자의 위치에 따라 추론됩니다.",
                    },
                },
                "required": ["location", "unit"],         #필수 파라미터 지정
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_n_day_weather_forecast",
            "description": "현재 날짜부터 N일 간의 날씨 예보를 가져옵니다",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "도시 또는 지역을 입력하세요, 예: 서울, 충청도",
                    },
                    "unit": {
                        "type": "string",
                        "enum": ["celsius", "fahrenheit"],
                        "description": "사용할 온도 단위를 입력하세요. 사용자의 위치에 따라 추론됩니다.",
                    },
                    "num_days": {
                        "type": "integer",
                        "description": "예보할 일수를 입력하세요",
                    }
                },
                "required": ["location", "unit", "num_days"]
            },
        }
    },
]

#함수 구현 (원래는 날씨 API호출 해야하지만 편의상 랜덤하게 지정)
import random

def get_n_day_weather_forecast(location, unit, num_days):

    weather_conditions = ["맑음", "흐림", "비", "눈", "안개", "폭풍"]
    forecasts = []

    for day in range(1, num_days+1):
        weather = random.choice(weather_conditions)
        temperature = round(random.uniform(-10, 35), 1) if unit == "celsius" else round(random.uniform(14, 95), 1)
        forecasts.append({"일차": day, "날씨": weather, "기온": temperature, "단위": unit})

    return str(forecasts)

def get_current_weather(location, unit):

    weather_conditions = ["맑음", "흐림", "비", "눈", "안개", "폭풍"]
    weather = random.choice(weather_conditions)
    temperature = round(random.uniform(-10, 35), 1) if unit == "celsius" else round(random.uniform(14, 95), 1)

    return str({"날씨": weather, "기온": temperature, "단위": unit})

# 예시 호출
print(get_n_day_weather_forecast("서울", "celsius", 5))
print(get_current_weather("대전", "fahrenheit"))


+ 프롬프트에 따라 어떤 Tool이 호출되는지 확인

In [ ]:

messages = []

messages.append({"role": "system", "content": "사용할 함수의 값에 대해 추측하지 마세요. 사용자의 요청이 모호한 경우 명확히 물어보세요."})
messages.append({"role": "user", "content": "오늘 날씨 어때?"})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
print(assistant_message)


In [ ]:
messages = []
messages.append({"role": "system", "content": "사용할 함수의 값에 대해 추측하지 마세요. 사용자의 요청이 모호한 경우 명확히 물어보세요."})
messages.append({"role": "user", "content": "서울 날씨 알고싶어."})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
messages.append(assistant_message)
print(assistant_message)


In [ ]:
messages = []
messages.append({"role": "system", "content": "사용할 함수의 값에 대해 추측하지 마세요. 사용자의 요청이 모호한 경우 명확히 물어보세요."})
messages.append({"role": "user", "content": "대전 5일간 날씨는 어때?"})
chat_response = chat_completion_request(
    messages, tools=tools
)
assistant_message = chat_response.choices[0].message
messages.append(assistant_message)
print(assistant_message)


+ 날씨 정보를 제공하는 챗봇

In [ ]:
from openai import OpenAI
client = OpenAI(api_key=OPENAI_API_KEY)

messages = [

    {"role": "system", "content": "당신은 사용자 질문에 답변하는 챗봇입니다."},
    {"role": "system", "content": "사용할 함수의 값에 대해 추측하지 마세요. 사용자의 요청이 모호한 경우 명확히 물어보세요."},
    {"role": "system", "content": "함수 호출이 아닌 답변의 경우 사용자가 읽기 쉽도록 마크다운 형태로 정리해서 출력하세요."},
]


# '종료' 입력 전까지 대화
user_input = ''
while True:
    user_input = input("You: ")
    if "종료" in user_input : break
    messages.append({"role": "user", "content": user_input})

    chat_response = chat_completion_request(messages, tools)

    # 1단계: 함수 호출로 이어질 수 있는 내용을 포함한 프롬프트
    # 이 경우, 모델은 사용자가 요청한 질문이 설정한 도구 설명에 부합되는지 확인 후 해당 함수 호출.
    assistant_message = chat_response.choices[0].message
    messages.append(assistant_message)

    # 2단계: 모델의 응답에 도구 호출이 포함되어 있는지 확인합니다.
    if assistant_message.tool_calls :
        for msg in assistant_message.tool_calls :
          # 3단계: 함수를 호출하고 결과를 얻습니다. 결과를 messages 리스트에 추가합니다.
          if "function" in msg.type:
              tool_id = msg.id  #함수 이름
              function_call = msg.function #함수 정보 가져오기
              function_name = function_call.name  #함수 이름
              function_args = json.loads(function_call.arguments) #함수 파라미터

              # 실제 함수 호출
              if function_name == "get_n_day_weather_forecast":
                  result = get_n_day_weather_forecast(**function_args)
                  # print("get_n_day_weather_forecast 호출했습니다.", result)

              elif function_name == "get_current_weather":
                  result = get_current_weather(**function_args)
                  # print("get_current_weather 호출했습니다.", result)
              messages.append({
                  "role": "tool",
                  "tool_call_id": tool_id,
                  "name": function_name,
                  "content": result
              })
        # 4단계: 함수 응답이 추가된 messages 리스트로 GPT API를 호출합니다.
        # tool_calls이 호출되면 반드시 해당 id에 대한 응답이 필요합니다.
        chat_response = chat_completion_request(messages)

    #결과 출력
    assistant_reply = chat_response.choices[0].message.content
    messages.append({"role": "assistant", "content": assistant_reply})
    print(f"Assistant: {assistant_reply}")

---